<a href="https://colab.research.google.com/github/welsakka/ArabicEnglishLectureModel/blob/main/Whisper_Arabic_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install

In [1]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796907 sha256=c45e2fd41175987729ab82e2258a1c977ee93ff63e2713c060e7e3d3c6447b1b
  Stored in directory: /root/.cache/pip/wheels/b2/13/5f/fe8245f6dc59df505879da4b2129932e342f02a80e6b87f27d
Successfully built openai-whisper


## Load custom model to use for Whisper

In [2]:
import whisper
import re
import torch
from google.colab import drive
# Mount google drive to output checkpoint file
drive.mount('/content/drive')

def hf_to_whisper_states(text):
    text = re.sub('.layers.', '.blocks.', text)
    text = re.sub('.self_attn.', '.attn.', text)
    text = re.sub('.q_proj.', '.query.', text)
    text = re.sub('.k_proj.', '.key.', text)
    text = re.sub('.v_proj.', '.value.', text)
    text = re.sub('.out_proj.', '.out.', text)
    text = re.sub('.fc1.', '.mlp.0.', text)
    text = re.sub('.fc2.', '.mlp.2.', text)
    text = re.sub('.fc3.', '.mlp.3.', text)
    text = re.sub('.fc3.', '.mlp.3.', text)
    text = re.sub('.encoder_attn.', '.cross_attn.', text)
    text = re.sub('.cross_attn.ln.', '.cross_attn_ln.', text)
    text = re.sub('.embed_positions.weight', '.positional_embedding', text)
    text = re.sub('.embed_tokens.', '.token_embedding.', text)
    text = re.sub('model.', '', text)
    text = re.sub('attn.layer_norm.', 'attn_ln.', text)
    text = re.sub('.final_layer_norm.', '.mlp_ln.', text)
    text = re.sub('encoder.layer_norm.', 'encoder.ln_post.', text)
    text = re.sub('decoder.layer_norm.', 'decoder.ln.', text)
    return text

# Load HF Model
hf_state_dict = torch.load("/content/drive/MyDrive/AI_output/checkpoint-4000/pytorch_model.bin")    # pytorch_model.bin file

# Rename layers
for key in list(hf_state_dict.keys())[:]:
    new_key = hf_to_whisper_states(key)
    hf_state_dict[new_key] = hf_state_dict.pop(key)

#pop unneeded keys
hf_state_dict.pop('proj_out.weight')

# Init Whisper Model and replace model weights
whisper_model = whisper.load_model('base')
whisper_model.load_state_dict(hf_state_dict)

Mounted at /content/drive


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 134MiB/s]


 bi Allahi fughi waas'i yu kan do do that


## Test Model with audio file and print results

In [ ]:
result = whisper_model.transcribe("/content/drive/MyDrive/AI Training/Arabic transliteration/mmtest.mp3")
print(result["text"])